In [1]:
import os 
#from fog.code.utils.utils import *

import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DoubleType
from pyspark.sql import SparkSession
from google.cloud import storage

# import Spark stuff
from pyspark import SparkFiles
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
spark = SparkSession.builder.appName('defog-labeled').getOrCreate()

In [3]:
defog_path = "parkinsons_data/train/processed/defog_tasks_lagging"
top_bucket_name = "msca-bdp-student-gcs"

In [4]:
def feed_files(top_bucket_name, prefix, suffix):
    client = storage.Client()
    blobs = client.list_blobs(top_bucket_name, prefix=prefix)

    processed = None

    for i, blob in enumerate(blobs):
        print(blob.name)
        if blob.name.endswith(suffix):
            
            if suffix == ".parquet":
                df = spark.read.parquet(f"gs://{top_bucket_name}/{blob.name}")
            elif suffix == ".csv":
                df = spark.read.csv(f"gs://{top_bucket_name}/{blob.name}")
            if processed is None:
                processed = df
            else:
                processed = processed.union(df)
    return processed



In [5]:
defog = feed_files(top_bucket_name, defog_path, ".parquet")

parkinsons_data/train/processed/defog_tasks_lagging/
parkinsons_data/train/processed/defog_tasks_lagging/_SUCCESS
parkinsons_data/train/processed/defog_tasks_lagging/part-00000-eae2acef-cda6-49a5-b43f-ded309b7bad5-c000.snappy.parquet


parkinsons_data/train/processed/defog_tasks_lagging/part-00001-eae2acef-cda6-49a5-b43f-ded309b7bad5-c000.snappy.parquet


parkinsons_data/train/processed/defog_tasks_lagging/part-00002-eae2acef-cda6-49a5-b43f-ded309b7bad5-c000.snappy.parquet
parkinsons_data/train/processed/defog_tasks_lagging/part-00003-eae2acef-cda6-49a5-b43f-ded309b7bad5-c000.snappy.parquet
parkinsons_data/train/processed/defog_tasks_lagging/part-00004-eae2acef-cda6-49a5-b43f-ded309b7bad5-c000.snappy.parquet
parkinsons_data/train/processed/defog_tasks_lagging/part-00005-eae2acef-cda6-49a5-b43f-ded309b7bad5-c000.snappy.parquet
parkinsons_data/train/processed/defog_tasks_lagging/part-00006-eae2acef-cda6-49a5-b43f-ded309b7bad5-c000.snappy.parquet
parkinsons_data/train/processed/defog_tasks_lagging/part-00007-eae2acef-cda6-49a5-b43f-ded309b7bad5-c000.snappy.parquet
parkinsons_data/train/processed/defog_tasks_lagging/part-00008-eae2acef-cda6-49a5-b43f-ded309b7bad5-c000.snappy.parquet
parkinsons_data/train/processed/defog_tasks_lagging/part-00009-eae2acef-cda6-49a5-b43f-ded309b7bad5-c000.snappy.parquet
parkinsons_data/train/processed/defog_ta

In [6]:
# drop Test, Visit, Medication cols because they were for tdcsfog
defog = defog.drop("Test", "Visit", "Medication")

# keep only when valid = True
defog = defog.filter(F.col("Valid")==True).filter(F.col("Task")==True)

# convert Time to float
defog = defog.withColumn("Time", F.col("Time").cast("float"))
defog.printSchema()

root
 |-- Subject: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- Time: float (nullable = true)
 |-- AccV: float (nullable = true)
 |-- AccML: float (nullable = true)
 |-- AccAP: float (nullable = true)
 |-- StartHesitation: integer (nullable = true)
 |-- Turn: integer (nullable = true)
 |-- Walking: integer (nullable = true)
 |-- Valid: boolean (nullable = true)
 |-- Task: boolean (nullable = true)
 |-- SourceDefog: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- YearsSinceDx: integer (nullable = true)
 |-- UPDRSIII_On: integer (nullable = true)
 |-- UPDRSIII_Off: integer (nullable = true)
 |-- NFOGQ: integer (nullable = true)
 |-- TimeSeconds: double (nullable = true)
 |-- Begin: double (nullable = true)
 |-- End: double (nullable = true)
 |-- TaskType: string (nullable = true)
 |-- MB9: integer (nullable = true)
 |-- Rest1: integer (nullable = true)
 |-- MB6-L: integer (nullable = true)
 |-- MB6-R: integer (nulla

In [7]:
# create sex binary cols
defog = defog.withColumn('SexInd', 
                  F.when((F.col("Sex") == "F"), 1) \
                    .when((F.col("Sex") == "M"), 0)
                  )

In [8]:
# convert task types to multilabels
# keep null tasks for now
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='TaskType', outputCol='TaskTypeInd', handleInvalid="keep")
model = indexer.fit(defog)
defog = model.transform(defog)

# use model.labels to see the order of the labels

In [9]:
# remove unnecessary cols
defog = defog.drop("Subject", "Sex", "Id", "TaskType")

In [10]:
defog.groupBy("target").count().show()

+------+-------+
|target|  count|
+------+-------+
|     1|     88|
|     3|  70521|
|     2| 414380|
|     0|3626334|
+------+-------+



In [11]:
# feature selection
# ignore standardized for now
def vectorize():
    """
    Creates vectorized dataframe.
    """
    """
    feature_cols = ["AccV", "AccML", "AccAP", "TaskTypeInd",
        "AccV_lag1", "AccV_lag2", "AccV_lag3", "AccV_lag4", "AccV_lag5",
        "AccV_lag6", "AccV_lag7", "AccV_lag8", "AccV_lag9", "AccV_lag10",
        "AccML_lag1", "AccML_lag2", "AccML_lag3", "AccML_lag4", "AccML_lag5",
        "AccML_lag6", "AccML_lag7", "AccML_lag8", "AccML_lag9", "AccML_lag10",
        "AccAP_lag1", "AccAP_lag2", "AccAP_lag3", "AccAP_lag4", "AccAP_lag5",
        "AccAP_lag6", "AccAP_lag7", "AccAP_lag8", "AccAP_lag9", "AccAP_lag10",
        "prediction4"]
    """
    #feature_cols = ['standardized'] + ["TaskTypeInd", "prediction4"]
    label_cols = ['target', 'StartHesitation', 'Turn', 'Walking']
    feature_cols = ["AccV", "AccML", "AccAP", "TaskTypeInd", "prediction4"]

    defog_sel = defog.select(feature_cols + label_cols)
    assembler = VectorAssembler(inputCols=feature_cols, outputCol='features', handleInvalid="skip")
    vectorized = assembler.transform(defog_sel)
    return vectorized

In [12]:
def run_logreg(train, test, labelCol):
    """
    Run the logistic regression model.
    """
    # create model
    lgr = LogisticRegression(maxIter=10, featuresCol = 'features', labelCol=labelCol)

    # fit model
    lgrm = lgr.fit(train)

    # make predictions
    predictions = lgrm.transform(test)

    return predictions


def get_f1(precision, recall):
    """
    Calculate f1 score.
    """
    if precision + recall == 0:
        return 0
    return 2 * ((precision * recall)/(precision + recall))

def get_accuracy(labels):
    """
    Calculate the accuracy for each label by creating a confusion matrix.
    """
    accs = {}

    preds_labels = predictions.select(F.col("prediction").cast(DoubleType()), F.col("target").cast(DoubleType())).orderBy('prediction')

    preds_labels = preds_labels.select(['prediction','target'])
    metrics = MulticlassMetrics(preds_labels.rdd.map(tuple))
    matrix = metrics.confusionMatrix().toArray()
    for label in labels:
        TN = 0
        FN = 0
        TP = 0
        FP = 0
        for i, row in enumerate(matrix):
            for j, col in enumerate(row):
                if i != label and j != label:
                    TN += matrix[i, j]
                elif i != label and j == label:
                    FN += matrix[i, j]
                elif i == label and j == label:
                    TP += matrix[i, j]
                elif i == label and j != label:
                    FP += matrix[i, j]
        print(f"label: {label}, TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
        if (TP + TN + FP + FN) == 0:
            acc = 0
        else:
            acc = (TP + TN)/(TP + TN + FP + FN)
        accs[label] = acc
    return accs

def evaluate(predictions, labels):
    """
    Evaluate model performance by computing accuracy and f1 scores.
    """
    evaluator = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction")
    print("Overall")
    print("Accuracy", evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
    weighted_precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
    weighted_recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
    weighted_f1 = get_f1(weighted_precision, weighted_recall)
    print("F1", weighted_f1)
    
    accs = get_accuracy(labels)
    f1_dict = {}
    for label in labels:
        print(label)
        precision = evaluator.evaluate(predictions, {evaluator.metricLabel: label, evaluator.metricName: "precisionByLabel"})
        recall = evaluator.evaluate(predictions, {evaluator.metricLabel: label, evaluator.metricName: "recallByLabel"})
        f1 = get_f1(precision, recall) 
        print("Accuracy", accs[label])
        print("F1", f1)
        f1_dict[label] = f1

    return evaluator, accs, f1_dict

Lag features, not standardized:  
> Accuracy 0.8811464693942825  
F1 0.8278086328796053

No lag, standardized:  
>Accuracy 0.8818123580030658  
F1 0.8396314248066408  

No lag, unstandardized:  
>Accuracy 0.8819386763598863  
F1 0.8404164161360337  

Use unstandardized, no lag


In [16]:
# no lag, unstandardized
vectorized = vectorize()
train, test = vectorized.randomSplit([0.8, 0.2],0.0)
predictions = run_logreg(train, test, 'target')

23/05/22 17:28:52 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/22 17:28:52 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [17]:
evaluator = evaluate(predictions, [0.0, 1.0, 2.0, 3.0])

Overall


Accuracy 0.8813211100183447


F1 0.8564489026360321


label: 0.0, TP: 718062.0, TN: 6900.0, FP: 6992.0, FN: 90627.0
label: 1.0, TP: 0.0, TN: 822563.0, FP: 18.0, FN: 0.0
label: 2.0, TP: 6896.0, TN: 732260.0, FP: 76429.0, FN: 6996.0
label: 3.0, TP: 0.0, TN: 808397.0, FP: 14184.0, FN: 0.0
0.0


Accuracy 0.8813259727613451
F1 0.9364448664002094
1.0


Accuracy 0.9999781176564982
F1 0
2.0


Accuracy 0.8985814162982125
F1 0.14153041472756475
3.0


Accuracy 0.9827567133206335
F1 0


In [18]:
predictions.select('prediction').groupBy('prediction').count().show()

+----------+------+
|prediction| count|
+----------+------+
|       0.0|809718|
|       2.0| 13813|
+----------+------+



Our algorithm can't predict labels 1 and 3.
We will try to treat it as a binary classification problem.  

Try classifying label 1 = Start Hesitation

In [19]:
vectorized.groupBy('StartHesitation').count().show()

+---------------+-------+
|StartHesitation|  count|
+---------------+-------+
|              1|     88|
|              0|4111235|
+---------------+-------+



In [20]:
# binary target=1
predictions = run_logreg(train, test, 'StartHesitation')
evaluate(predictions, [0.0, 1.0])

Overall


Accuracy 0.8821894436581754


F1 0.8269711820180207


label: 0.0, TP: 724933.0, TN: 0.0, FP: 0.0, FN: 96810.0
label: 1.0, TP: 0.0, TN: 821723.0, FP: 20.0, FN: 0.0
0.0


Accuracy 0.8821894436581754
F1 0.9374077052983302
1.0


Accuracy 0.9999756614902713
F1 0


(MulticlassClassificationEvaluator_8406fdb4c76f,
 {0.0: 0.8821894436581754, 1.0: 0.9999756614902713},
 {0.0: 0.9374077052983302, 1.0: 0})

In [21]:
# show predicted labels
predictions.groupBy("prediction").count().show()

+----------+------+
|prediction| count|
+----------+------+
|       0.0|821743|
+----------+------+



Treating label 1 as a binary problem doesn't improve classification. We will try to undersample the data instead.

In [15]:
def resample(large_dataframe, ratio, class_field, base_class):
    """
    Resamples non-minority label by a chosen factor
    """
    pos = large_dataframe.filter(large_dataframe[class_field] != base_class)
    neg = large_dataframe.filter(large_dataframe[class_field] == base_class)
    total_pos = pos.count()
    total_neg = neg.count()
    fraction=float(total_pos*ratio)/float(total_neg)
    sampled = neg.sample(False,fraction)
    
    return sampled.union(pos)
sampled = resample(vectorized, 6, 'StartHesitation', 0)
sampled.select("StartHesitation").groupBy("StartHesitation").count().show()

+---------------+-----+
|StartHesitation|count|
+---------------+-----+
|              1|   88|
|              0|  526|
+---------------+-----+



In [23]:
# binary target=1, undersampled majority
train, test = sampled.randomSplit([0.8, 0.2],0.0)
predictions = run_logreg(train, test, 'StartHesitation')
evaluate(predictions, [0.0, 1.0])

Overall


Accuracy 0.717741935483871


F1 0.6161746804625685


label: 0.0, TP: 89.0, TN: 1.0, FP: 2.0, FN: 32.0
label: 1.0, TP: 0.0, TN: 103.0, FP: 18.0, FN: 3.0
0.0


Accuracy 0.7258064516129032
F1 0.8476190476190476
1.0


Accuracy 0.8306451612903226
F1 0


(MulticlassClassificationEvaluator_b097261f9f2d,
 {0.0: 0.7258064516129032, 1.0: 0.8306451612903226},
 {0.0: 0.8476190476190476, 1.0: 0})

In [25]:
predictions.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|  114|
|       1.0|    6|
+----------+-----+



This is better since we're at least trying to predict the label, but we've had to severely undersample the data.

Now treat label 3=walking as binary

In [26]:
vectorized.groupBy('Walking').count().show()

+-------+-------+
|Walking|  count|
+-------+-------+
|      1|  70521|
|      0|4040802|
+-------+-------+



In [27]:
# binary target=3
train, test = vectorized.randomSplit([0.8, 0.2],0.0)
predictions = run_logreg(train, test, 'Walking')
evals = evaluate(predictions, [0.0, 1.0])

Overall


Accuracy 0.8814391796967651


F1 0.8259069220697919


label: 0.0, TP: 724662.0, TN: 0.0, FP: 11.0, FN: 97462.0
label: 1.0, TP: 0.0, TN: 822103.0, FP: 21.0, FN: 11.0
0.0


Accuracy 0.8814391796967651
F1 0.9369839739797788
1.0


Accuracy 0.999961076952082
F1 0


In [28]:
predictions.groupBy("prediction").count().show()

+----------+------+
|prediction| count|
+----------+------+
|       0.0|822124|
|       1.0|    11|
+----------+------+



Now we try to undersampling

In [16]:
# undersample majority relative to "Walking"
sampled = resample(vectorized, 6, 'Walking', 0)
sampled.select("Walking").groupBy("Walking").count().show()

+-------+------+
|Walking| count|
+-------+------+
|      1| 70521|
|      0|423703|
+-------+------+



In [18]:
# binary target=3, undersampled majority
train, test = sampled.randomSplit([0.8, 0.2],0.0)
predictions = run_logreg(train, test, 'Walking')

23/05/22 17:59:34 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/22 17:59:34 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [19]:
evals = evaluate(predictions, [0.0, 1.0])

Overall


Accuracy 0.7694047787360384


F1 0.6686921174972204


label: 0.0, TP: 76051.0, TN: 2.0, FP: 34.0, FN: 23036.0
label: 1.0, TP: 0.0, TN: 99084.0, FP: 3.0, FN: 36.0
0.0


Accuracy 0.767258860203989
F1 0.8683008700020551
1.0


Accuracy 0.9996065494385763
F1 0


In [21]:
predictions.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|99087|
|       1.0|   36|
+----------+-----+



Even though some models produce higher accuracies and F1 scores, they may not be the most sensitive models because they end up not being able to predict all of our labels. This is likely due to our unbalanced dataset, that has a lot of rows with labels=0, 2 and a relatively small number with labels=1, 3. Some areas to explore in the future could be a combination of undersampling and oversampling, or appying a method such as SMOTE to generate new data points.